IMPORT


if you want want to use the notebook in google colabs you can use the code below, but if you want to use it locally you can skip the first step to mount the google drive.

In [81]:
#from google.colab import drive
#drive.mount('/content/drive')

petroleum data scraped from https://www.energy.gov.za/files/esources/petroleum/petroleum_arch.html
lending data scraped from https://www.fnb.co.za/rates/LendingRates.html

In [82]:
import pandas as pd
import numpy as np

from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

from xgboost import XGBRegressor

from lightgbm import LGBMRegressor

import math
import warnings
warnings.filterwarnings('ignore')

from statsmodels.tsa.filters.hp_filter import hpfilter

pd.options.display.max_columns = 2000
pd.options.display.max_rows = 2000

#put the your data folder path here
path = '/content/drive/MyDrive/submit'
cpi = pd.read_csv('CPI_Historic_Values_Zindi_May_23.csv')
petrol = pd.read_csv('petrolem.csv')
lending = pd.read_csv('historical_prime_lending_rates.csv')
lending = lending.ffill()
seed = 3

PIVOTING THE DATA


In [83]:
cpi_pivot = cpi.pivot(index = 'Month', columns = 'Category', values = 'Value').reset_index()#changing from a long format to a wide format,
cpi_pivot['Month'] = pd.to_datetime(cpi_pivot['Month'])
cpi_pivot = cpi_pivot.sort_values("Month").reset_index(drop=True)

In [84]:
cpi_pivot = cpi_pivot.drop(cpi_pivot.index[-1])

ADDING JUNE'S DATA MANUALLY FROM https://www.statssa.gov.za/publications/P0141/P0141June2023.pdf


ADD THE july ROW


In [85]:
date_str = '2023-05-31'
date_obj = pd.to_datetime(date_str)
new_row = pd.DataFrame({'Month': [date_obj]})
cpi_pivot = pd.concat([cpi_pivot, new_row]).reset_index(drop=True)

FEATURE ENGINNERING


the only feature i used is a 5 times lagging features of all the month's cpi's. nothing fancy other than that.

In [86]:
feats_to_lag = cpi_pivot.columns[1:].to_list()
for col in feats_to_lag:
    for i in range(1,3):
        cpi_pivot[f'prev_{i}_month_{col}'] = cpi_pivot[col].shift(i)
columns = list(cpi_pivot.columns)
columns = [item for item in columns if "Transport" not in item]

Add petrol data

In [87]:
petrol['Month'] = pd.to_datetime(petrol['Month'])
petrol = petrol.sort_values("Month").reset_index(drop=True)
feats_to_lag = petrol.columns[1:].to_list()
for col in feats_to_lag:
    for i in range(1,3):
        petrol[f'prev_{i}_month_{col}'] = petrol[col].shift(i)
start_date = pd.to_datetime('2022-01-31')
end_date = pd.to_datetime('2023-05-31')
petrol = petrol[(petrol['Month'] >= start_date) & (petrol['Month'] <= end_date)]
cpi_pivot = pd.merge(cpi_pivot, petrol, on='Month', suffixes=('_df1', '_df2'))

In [88]:
lending['Interest_per_annum'] = lending['Interest_per_annum'].str.rstrip('%').astype(float).astype(int)
lending['Month'] = pd.to_datetime(lending['Month'])
lending = lending.sort_values("Month").reset_index(drop=True)
feats_to_lag = lending.columns[1:].to_list()
for col in feats_to_lag:
    for i in range(1,3):
        lending[f'prev_{i}_month_{col}'] = lending[col].shift(i)
start_date = pd.to_datetime('2022-01-31')
end_date = pd.to_datetime('2023-05-31')
lending = lending[(lending['Month'] >= start_date) & (lending['Month'] <= end_date)]
cpi_pivot = pd.merge(cpi_pivot, lending, on='Month', suffixes=('_df1', '_df2'))

In [89]:
petcol = list(petrol.columns)
search_strings = ["petrol95" ,"randuS_exchenge" ]
petcol = [item for item in petcol if all(search_str not in item for search_str in search_strings)]
petcol = petcol[1:]

HANDLE MISSING DATA


In [90]:
cpi_pivot = cpi_pivot.drop(0)
cpi_pivot = cpi_pivot.bfill()

TRAIN AND VALIDATION


In [91]:
train = cpi_pivot[cpi_pivot['Month'] != "2023-05-31"]
test = cpi_pivot[cpi_pivot['Month'] == "2023-05-31"]

training_set = train[train['Month']!= '2023-04-30']
validation_set = train[train['Month']== '2023-04-30']

train.shape, test.shape, training_set.shape, validation_set.shape

((15, 61), (1, 61), (14, 61), (1, 61))

MODELING


In [92]:
target_cols = ['Alcoholic beverages and tobacco', 'Clothing and footwear',
       'Communication', 'Education', 'Food and non-alcoholic beverages',
       'Headline_CPI', 'Health', 'Household contents and services',
       'Housing and utilities', 'Miscellaneous goods and services',
       'Recreation and culture', 'Restaurants and hotels ', 'Transport']

#if you add additional data sources that have no value in the predicting month , drop it, now that you have their lags
features= [col for col in train.columns if col not in target_cols + ['Month']]
featurest= [col for col in train.columns if col not in columns + ['Transport']]

X_train = training_set[features]
X_traint = training_set[featurest]
y_train = training_set[target_cols]

X_val = validation_set[features]
X_valt = validation_set[featurest]
y_val = validation_set[target_cols]

l_models = {}

y_predl = []

rmsel_dict = {} 

scaler = MinMaxScaler()

#training
for target_col in target_cols:
    if target_col == "Transport":
        l_model = Lasso(alpha=0.1, random_state=seed)
        #l_model = XGBRegressor(seed=seed)
        X_train_scaled = scaler.fit_transform(X_traint)

        l_model.fit(X_train_scaled, y_train[target_col])

        l_models[target_col] = l_model
        
        l_model = l_models[target_col]

        X_val_scaled = scaler.transform(X_valt)

        y_pred_coll = l_model.predict(X_val_scaled)

        rmsel_col = np.sqrt(mean_squared_error(y_pred_coll, y_val[target_col]))
        rmsel_dict[target_col] = rmsel_col 

        y_predl.append(y_pred_coll)
    else:
        l_model = Lasso(alpha=0.06, random_state=seed)
        X_train_scaled = scaler.fit_transform(X_train)

        l_model.fit(X_train_scaled, y_train[target_col])

        l_models[target_col] = l_model
        
        l_model = l_models[target_col]

        X_val_scaled = scaler.transform(X_val)

        y_pred_coll = l_model.predict(X_val_scaled)

        rmsel_col = np.sqrt(mean_squared_error(y_pred_coll, y_val[target_col]))
        rmsel_dict[target_col] = rmsel_col 

        y_predl.append(y_pred_coll)

# scoring
y_predl = np.array(y_predl).T
y_predl[:,5][0] = ((6.26 * y_predl[:,0][0]) + (3.65 * y_predl[:,1][0]) + (2.42 * y_predl[:,2][0]) 
                   + (2.62 * y_predl[:,3][0]) + (17.14 * y_predl[:,4][0]) + (1.44 * y_predl[:,6][0]) + 
                   (4.37 * y_predl[:,7][0]) 
                   + (24.49 * y_predl[:,8][0]) + (14.81 * y_predl[:,9][0]) + (5.2 * y_predl[:,10][0]) + 
                   (3.25 * y_predl[:,11][0]) + (14.35 * y_predl[:,12][0]))/100

dfl = pd.DataFrame({'y_pred': y_predl.flatten(), 'y_val': y_val.values.flatten()})

# Print RMSE for each target column
for target_col in target_cols:
    print(f'RMSE of Lasso Regression for {target_col}: {rmsel_dict[target_col]}')

# Calculate the average RMSE across all target columns
average_rmse = np.mean(list(rmsel_dict.values()))
print(f'Average RMSE of Lasso Regression: {average_rmse}')

RMSE of Lasso Regression for Alcoholic beverages and tobacco: 0.7783608118173646
RMSE of Lasso Regression for Clothing and footwear: 0.36297313292125466
RMSE of Lasso Regression for Communication: 0.13480842938885473
RMSE of Lasso Regression for Education: 1.55368034955805
RMSE of Lasso Regression for Food and non-alcoholic beverages: 0.11368177919271716
RMSE of Lasso Regression for Headline_CPI: 0.16030512826014842
RMSE of Lasso Regression for Health: 1.8512642900123808
RMSE of Lasso Regression for Household contents and services: 0.9127889759354844
RMSE of Lasso Regression for Housing and utilities: 0.015831217758261573
RMSE of Lasso Regression for Miscellaneous goods and services: 2.2938993685350084
RMSE of Lasso Regression for Recreation and culture: 0.9695469083613091
RMSE of Lasso Regression for Restaurants and hotels : 0.48019173927109193
RMSE of Lasso Regression for Transport: 0.10347425490687101
Average RMSE of Lasso Regression: 0.7485235681475998


In [93]:
X_train = train[features]
X_traint = train[featurest]
y_train = train[target_cols]

X_val = test[features]
X_valt = test[featurest]
y_val = test[target_cols]

l_models = {}

y_predl = []

scaler = MinMaxScaler()

#training
for target_col in target_cols:
    if target_col == "Transport":
        l_model = Lasso(alpha=0.1, random_state=seed)
        #l_model = XGBRegressor(seed=seed)
        X_train_scaled = scaler.fit_transform(X_traint)

        l_model.fit(X_train_scaled, y_train[target_col])

        l_models[target_col] = l_model
        l_model = l_models[target_col]

        X_val_scaled = scaler.transform(X_valt)

        y_pred_coll = l_model.predict(X_val_scaled)

        y_predl.append(y_pred_coll)
    else:
        l_model = Lasso(alpha=0.06, random_state=seed)

        X_train_scaled = scaler.fit_transform(X_train)

        l_model.fit(X_train_scaled, y_train[target_col])

        l_models[target_col] = l_model
        l_model = l_models[target_col]

        X_val_scaled = scaler.transform(X_val)

        y_pred_coll = l_model.predict(X_val_scaled)

        y_predl.append(y_pred_coll)

y_predl = np.array(y_predl).T
y_predl[:,5][0] = ((6.26 * y_predl[:,0][0]) + (3.65 * y_predl[:,1][0]) + (2.42 * y_predl[:,2][0]) 
                   + (2.62 * y_predl[:,3][0]) + (17.14 * y_predl[:,4][0]) + (1.44 * y_predl[:,6][0]) + 
                   (4.37 * y_predl[:,7][0]) 
                   + (24.49 * y_predl[:,8][0]) + (14.81 * y_predl[:,9][0]) + (5.2 * y_predl[:,10][0]) + 
                   (3.25 * y_predl[:,11][0]) + (14.35 * y_predl[:,12][0]))/100
y_predl

array([[110.61297877, 104.06309147,  99.69333333, 109.34759486,
        121.45739881, 110.3276712 , 109.9568288 , 108.88689238,
        105.47714114, 108.7864583 , 104.39515492, 110.01691534,
        113.04008648]])

SUBMITION


In [94]:
def prepSub(y_pred:list, target_cols: list, test, prefix:str):
    sub_df = pd.DataFrame(y_pred, columns=target_cols)
    sub_df['Month'] = test['Month']

    sub_df.set_index('Month', inplace=True)
    sub_df.columns = [prefix+'_' + col.lower().replace('_', ' ').strip() for col in sub_df.columns]
    sub_df.rename(columns= {f"{prefix}_headline cpi": f"{prefix}_headline CPI"}, inplace=True)

    sub_df = pd.melt(sub_df.reset_index(), id_vars= ['Month'], var_name= 'ID', value_name= 'Value')

    return sub_df[['ID', 'Value']]

In [95]:
sub = prepSub(y_predl, target_cols, test, 'May')
sub.to_csv(f'lasso06_with_lag355_historic_petrol_lending.csv', index=False)